In [1]:
import pandas as pd
import numpy as np
import os
from env import host, user, password
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import env
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.impute import SimpleImputer
import scipy
import sklearn.linear_model
import sklearn.preprocessing
import warnings
warnings.filterwarnings("ignore")
import wrangle
import viz_explore
import scipy.stats as stats
import brian_model
import cluster_model
import brian_feature_engineering

from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.linear_model
import sklearn.preprocessing
import warnings

In [2]:
df, train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, \
train_scaled, X_train_scaled, y_train_scaled, validate_scaled, X_validate_scaled, \
y_validate_scaled, test_scaled, X_test_scaled, y_test_scaled \
= wrangle.wrangle_zillow()


In [3]:
X_train_scaled.columns

Index(['bathrooms', 'bedrooms', 'condition', 'sq_ft', 'full_baths', 'latitude',
       'longitude', 'lot_size', 'rooms', 'structure_value', 'tax_value',
       'year_assessed', 'land_value', 'tax_amount', 'age',
       'sq_ft_per_bathroom', 'sq_ft_per_bedroom', 'sq_ft_per_room',
       'has_half_bath', 'tax_rate', 'price_per_sq_ft', 'Los_Angeles', 'Orange',
       'Ventura'],
      dtype='object')

In [4]:
# This cell runs my function in feature_engineering.py that selects for the top 'k' number 
# of features
#f_feature = brian_feature_engineering.select_kbest(X_train_scaled,y_train_scaled, 5)
#f_feature

In [5]:
_long_sf_train = X_train_scaled[['sq_ft', 'tax_rate', 'longitude']]
_long_sf_validate = X_validate_scaled[['sq_ft', 'tax_rate', 'longitude']]
_long_sf_test = X_test_scaled[['sq_ft', 'tax_rate', 'longitude']]

In [6]:
# sklearn implementation of KMeans

#define the thing
kmeans_long_sf = KMeans(n_clusters=4, random_state = 123, verbose = 0)

# fit the thing
kmeans_long_sf.fit(_long_sf_train)

# Use (predict using) the thing 
kmeans_long_sf.predict(_long_sf_train)

array([2, 1, 0, ..., 0, 3, 2], dtype=int32)

##

In [7]:
X_train_scaled['log_sf_cluster'] = kmeans_long_sf.predict(_long_sf_train)


In [8]:
X_train_scaled['log_sf_cluster'] = np.where(X_train_scaled.log_sf_cluster == 0,'small_low_to_medtax',np.where(X_train_scaled.log_sf_cluster == 1,'larger_lo_to_medtax',\
        np.where(X_train_scaled.log_sf_cluster == 2, 'all_size_low_to_midtax',\
                 'all_size_mid_to_hg_tax')))

In [9]:
X_validate_scaled['log_sf_cluster'] = kmeans_long_sf.predict(_long_sf_validate)
X_validate_scaled['log_sf_cluster'] = np.where(X_validate_scaled.log_sf_cluster == 0,'small_low_to_medtax',np.where(X_validate_scaled.log_sf_cluster == 1,'larger_lo_to_medtax',\
        np.where(X_validate_scaled.log_sf_cluster == 2, 'all_size_low_to_midtax',\
                 'all_size_mid_to_hg_tax')))

In [10]:
X_test_scaled['log_sf_cluster'] = kmeans_long_sf.predict(_long_sf_test)
X_test_scaled['log_sf_cluster'] = np.where(X_test_scaled.log_sf_cluster == 0,'small_low_to_medtax',np.where(X_test_scaled.log_sf_cluster == 1,'larger_lo_to_medtax',\
        np.where(X_test_scaled.log_sf_cluster == 2, 'all_size_low_to_midtax',\
                 'all_size_mid_to_hg_tax')))

In [11]:
# this is also in the cluster_model.py file

def encode_clusters(df,cluster):
    '''
    This is encoding the cluster column for later modelling; it drops the original column 
    once it has been encoded
    
    '''
    # ordinal encoder? sklearn.OrdinalEncoder

    # I had originally put the columns to be dummied inside the function. They are now in the inputs
    # cols_to_dummy = df['county']
    # df = pd.get_dummies(df, columns=['county'], dummy_na=False, drop_first=False)
        
    dummy_df = pd.get_dummies(df[cluster], dummy_na=False, drop_first=False)

    # Not requiring me to concatenate for some reason here.  
    df = pd.concat([df, dummy_df], axis = 1)
#     df = df.rename(columns={'county_Los_Angeles':'Los_Angeles','county_Orange':'Orange','county_Ventura':'Ventura'})
    df = df.drop(columns='log_sf_cluster')
    return df

In [12]:
X_train_scaled = encode_clusters(X_train_scaled,['log_sf_cluster'])

In [13]:
X_validate_scaled = encode_clusters(X_validate_scaled,['log_sf_cluster'])

In [14]:
X_test_scaled = encode_clusters(X_test_scaled,['log_sf_cluster'])

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
logerror_pred_mean = y_train_scaled.logerror.mean()
y_train_scaled['logerror_pred_mean'] = round(logerror_pred_mean, 5)
y_validate_scaled['logerror_pred_mean'] = round(logerror_pred_mean,5)
y_test_scaled['logerror_pred_mean'] = round(logerror_pred_mean,5)

In [17]:
rmse_train = mean_squared_error(y_train_scaled.logerror,
                                y_train_scaled.logerror_pred_mean) ** .5
rmse_validate = mean_squared_error(y_validate_scaled.logerror, y_validate_scaled.logerror_pred_mean) ** (0.5)

print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 5), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 5))

RMSE using Mean
Train/In-Sample:  0.01897 
Validate/Out-of-Sample:  0.01899


In [18]:
# taken in by the modeling function below and expanded on.
metric_df = pd.DataFrame(data=[
            {
                'model': 'mean_baseline', 
                'RMSE_train': rmse_train,
                'RMSE_validate': rmse_validate
                }
            ])
metric_df

,model,RMSE_train,RMSE_validate
0,mean_baseline,0.018968,0.018994


In [29]:
# function to compute adjusted R-squared
def adj_r2_score(predictors, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = predictors.shape[0]
    k = predictors.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# function to compute MAPE
def mape_score(targets, predictions):
    return np.mean(np.abs(targets - predictions) / targets) * 100


# function to compute different metrics to check performance of a regression model
def model_performance_regression(model, predictors, target):
    """
    Function to compute different metrics to check regression model performance

    model: regressor
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    # computing the actual prices by using the exponential function
    target = np.exp(target)
    pred = np.exp(pred)

    r2 = r2_score(target, pred)  # to compute R-squared
    adjr2 = adj_r2_score(predictors, target, pred)  # to compute adjusted R-squared
    rmse = np.sqrt(mean_squared_error(target, pred))  # to compute RMSE

        

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "R-squared": r2,
            "Adj. R-squared": adjr2
        },
        index=[0],
    )

    return df_perf

In [39]:
y_train_scaled.columns

Index(['logerror', 'logerror_pred_mean'], dtype='object')

In [44]:
# fitting a linear model
lin_reg_model = LinearRegression()
lin_reg_model.fit(X_train_scaled, y_train_scaled)

LinearRegression()

In [45]:
print('Train performance')
lin_reg_model_perf_train = model_performance_regression(lin_reg_model, X_train_scaled, y_train_scaled)
lin_reg_model_perf_train

Train performance


,RMSE,R-squared,Adj. R-squared
0,0.025645,0.001697,0.000434


In [46]:
print("Validate Performance:")
lin_reg_model_perf_test = model_performance_regression(lin_reg_model, X_validate_scaled, y_validate_scaled)
lin_reg_model_perf_test

Validate Performance:


,RMSE,R-squared,Adj. R-squared
0,0.025137,0.375639,0.373793


In [48]:
print("Test Performance:")
lin_reg_model2_perf_test = model_performance_regression(lin_reg_model, X_test_scaled, y_test_scaled)
lin_reg_model2_perf_test

 

Test Performance:


,RMSE,R-squared,Adj. R-squared
0,0.024777,0.376168,0.373953
